# **Logic**

We first calculate the ratio of output (fee) to the input (weight) for each transaction.
Let us call this metric as "efficiency".<br>
So, the task of the miner is to maximise his efficiency while adhering to the block limit.<br><br>
We sort the transactions in decreasing order of efficiency and then check each transaction from beginning<br><br>
Any transaction that doesnt have a parent is automatically confirmed.<br>
If any transaction has parent(s), then each parent transaction is checked.
From here, 3 cases arise:<br><br>
<ol>
<li> The parent(s) are already confirmed (transaction confirmed)
<li> The parent(s) exist but arent confirmed (transaction kept in probation*)
<li> The parent(s) do not exist (transaction not confirmed)
</ol>

<br><br>
*The parent(s) of that transaction may occur somewhere below that transaction, indicating that the parents' efficiency may be less.
If the parent is only a little "inefficient", then it may be picked up for confirmation, else the transaction itself is discarded. This  is why the transaction is kept in probation.

In [1]:
import pandas as pd
import numpy as np

In [3]:
file = pd.read_csv("/content/sample_data/mempool.csv", skiprows=1, names=["tx_id", "fee", "weight", "parents"])

In [4]:
# Adding the efficiency field so that mempool can be sorted by the same

efficiency = pd.Series([], dtype="object")

for i in range(len(file)):
  efficiency[i] = file["fee"][i]/file["weight"][i]

file.insert(4, "efficiency", efficiency)

In [9]:
sorted = file.sort_values(by=["efficiency"], ascending=False)

In [10]:
# global variables

confirmed = []
probation = []    # list of indices of transaction in probation
total_weight = 0
total_fee = 0

In [11]:
# Helper functions

def are_parents_null(index):
  return pd.isnull(sorted["parents"][index])

def are_parents_confirmed(index):
  # checking if parents appear in confirmed
  parents = sorted["parents"][index].split(";")
  for parent in parents:     
      if parent not in confirmed:
        return False
  return True

def exceeds_block(index):
  return total_weight + sorted["weight"][index] > 4000000

def check_probation():
  global total_fee, total_weight
  index_remove = []

  # checks if each parent is confirmed
  # while keeping an eye on the block limit
  for index in probation:
    if exceeds_block(index):
      continue

    if are_parents_confirmed(index):
      # the transaction is confirmed
      # all the confirmed tx's will be cleared in the next step
      index_remove.append(index)
      confirmed.append(sorted["tx_id"][index])
  
  for index in index_remove:
    # transaction is taken out of probation

    probation.remove(index)
  
  return [confirmed, probation, total_weight, total_fee]

def confirm(index):
  global total_fee, total_weight

  # confirm the transaction
  confirmed.append(sorted["tx_id"][index])
  total_fee += sorted["fee"][index]
  total_weight += sorted["weight"][index]

  return [confirmed, total_weight, total_fee]

In [12]:
for tx_index in range(len(sorted_file)):
  # checks if any tx in probation can be confirmed
  check_probation()

  if are_parents_null(tx_index):
    # parents doesn't exist

    if exceeds_block(tx_index):
      # go to the next tx entry if this one doesn't fit the block
      continue

    # but if this tx fits the block, confirm the transaction
    confirm(tx_index)

  else: 
    # parent does exist

    if are_parents_confirmed(tx_index):
      # parents are confirmed too

      if exceeds_block(tx_index):
        # move to the next tx if this one doesnt fit
        continue
      
      else:
        # this transaction fits the block. Confirm it
        confirm(tx_index)
        
    else:
      # one or more parents arent confirmed yet.
      # add to probation and move on

      probation.append(tx_index)
      continue

In [28]:
# print("length of confirmed list is ", len(confirmed))
# print("fee objained: ", total_fee)
# print("weight of block: ", total_weight)
with open("/content/sample_data/block.txt", "w") as f:
  for tx_id in confirmed:
    f.write(tx_id + "\n")
  